# Исседование займов на различные проекты платформы kiva.org

Для анализа предоставлены данные с платформы краудфандинговой платформы kiva.org за период с 2014-2017 год. Датасет содержит информацию о займах на различные проекты в 85 разных странах. 
Анализ проведен для выявления взаимосвязи характеристик со скоростью набора средств, в частности связана ли доля заёмщиков-женщин с основной религией страны, главных финансовых партнёров, зависимости сектора заявки от индекса бедности страны и в каких странах Kiva недостаточно представлена. 

1. [Изучение данных](#start)
2. [Обработка дубликатов](#1)
3. [Обработка аномалий](#2)
4. [Обработка типов данных](#3)
5. [Зависимость скорости сбора средств от суммы кредита](#4)
6. [Зависимость скорости сбора средств от сектора и деятельности](#5)
7. [Зависимость скорости сбора средств от партнера](#6)
8. [Зависимость скорости сбора средств от страны, в которой был выдан кредит](#7)
9. [Анализ заявок и индекса бедности](#8)
10. [Распределение женщин-заемщиков в разных странах и религиях](#9)
11. [Партнёрские отношения Kiva в регионах](#10)
12. [Общие выводы](#11)

In [ ]:
#загрузка данных
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter('ignore') 
import plotly.express as px 
from plotly import graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
df = pd.read_csv('...')

**Изучение данных.**
<a id="start"></a>

In [ ]:
df.hist(figsize=(10, 10))

In [ ]:
#рассмотрю типы данных
df.info()

**Выводы по выгрузке данных**

Как мы можем заметить, представленный датафрейм содержит информацию о характеристиках каждой заявки. Фрейм содержит 21 столбец, 15 из которых имеют тип данных object, 4 из которых имеют тип float64, и два столбца с данными типа int64. Также на гистограмме видны аномальные значения, которые позже будут исключены для получения более достоверного результата.

**Изучение данных**

In [ ]:
report = df.isna().sum().to_frame()
report = report.rename(columns = {0: 'missing_values'})
report['% of total'] = (report['missing_values'] / df.shape[0]).round(2)
report.sort_values(by = 'missing_values', ascending = False)

Больше всего отсутсвующих значений выявлено в столбце tags(27%), но эти пропуски невозможно заполнить синтетическим значением, а заполнение на основе анализа каждого поста потребует слишком много времени, поэтому, учитывая что этот столбец не будет учавствовать в исследовании, заполню пустые значения на 'unknown', создав так отдельную категорию.

In [ ]:
df['tags'] = df['tags'].fillna('unknown')
df['tags'].isna().sum()
#по аналогии действую с столбцом use 
df['use'] = df['use'].fillna('unknown')

Также значительное количетсво пропусков обнаружено в столбце region, однако цель исследования главным образом заключается в выявлении закономерностей на основе показателей страны, а не региона, поэтому на данный момент проуски можно оставить. По тем же причинам не корректировать country_code.

**Перехожу к обработке дубликатов**

<a id="1"></a>

In [ ]:
#проверяю наличие явных дубликатов
df.duplicated().sum()
df['id'].duplicated().sum()

Таблица не содержит явных дубликатов. Так как id это уникальный код заявки, то дублирование этого значение уже можно считать ошибкой в данных или дубликатом. 

**Приступаю к обработке аномалий**

так как нам для анализа нужна скорость сбора средств, аномально большие значения могут исказить результат и вывод, поэтому оставляю самые частые результаты. По аналогии обработаем все столбцы, проверяя процент оставшихся значений.
<a id="2"></a>

In [ ]:
df.hist(figsize=(7, 7))

In [ ]:
df_1 = df.query('0 < funded_amount < 2400')
df_1.shape[0] / df.shape[0]
df_1 = df_1.query('0 < loan_amount < 2400')
df_1.shape[0] / df.shape[0]
df_1['term_in_months'].hist(figsize=(5, 5), range=(0, 100))
df_1 = df_1.query('0 < term_in_months < 36')
df_1.shape[0] / df.shape[0]

Исключенные данные позволят сделать более реалистичные выводы, игнорируя аномально большие значения, при этом процент удаленных данных составляет меньше 10 процентов, что существенно не исказит результаты.

**Приступаю с обработке типов данных.**

In [ ]:
df_1.info()

In [ ]:
df_1['posted_time'] = pd.to_datetime(df_1['posted_time'], format='%Y-%m-%d %H:%M:%S%z')
df_1['disbursed_time'] = pd.to_datetime(df_1['disbursed_time'], format='%Y-%m-%d %H:%M:%S%z')
df_1['funded_time'] = pd.to_datetime(df_1['funded_time'], format='%Y-%m-%d %H:%M:%S%z')
df_1['date'] = pd.to_datetime(df_1['date'], format='%Y-%m-%d')

**Анализ скорости сбора средств**
<a id="3"></a>

Рассчитаю время, которое было затрачено на сбор средств по каждой заявке, выраженное в количестве дней.

In [ ]:
df_1['collect'] = (df_1['funded_time'] - df_1['posted_time']).dt.days
df_1['collect'].max()
df_1['collect'].hist(range=(0, 100))

Разделю заявки по времени сбора средств на 5 групп для большей наглядности. Выделю группу с заявками после 40 дней, потому что на гистограмме виден скачок в этой точке.

In [ ]:
def collect_group(row):
    collect = row['collect']
    if 1 > collect:
        return 'собраны за первый день'
    if 1 <= collect <= 7:
        return 'собраны за первую неделю'
    if 7 < collect <= 40:
        return 'собраны в течение 40 дня'
    if 40 < collect <= 60:
        return 'собраны в течение двух месяцев'
    if collect > 60 :
        return 'собраны более чем через два месяца'
df_1['collect_group'] = df_1.apply(collect_group, axis = 1)

In [ ]:
collect_group = df_1['collect_group'].value_counts().sort_values().reset_index()
collect_group

In [ ]:
fig = go.Figure(go.Funnel(
    y = collect_group['collect_group'],
    x = collect_group['count']
))

fig.update_layout(title="Скорость сбора средств")

fig.show()

Как мы видим из полученной таблицы и воронки, подавляющее большинство заявок было удволетворено в течение 40 дня (316590) или первой недели (226764). Реже необходимая сумма собирается в первы день(20136) или в течение двух месяцев(9822), но самое небольшое количество заявок, равное 2109 было собрано более чем через 2 месяца. Нужно рассмотреть, какие факторы влияют на скорость сбора средств.

**Зависимость скорости сбора средств от суммы кредита**
<a id="4"></a>

In [ ]:
df_1.pivot_table(index='collect', values='funded_amount', aggfunc='mean').plot(style='o')
df_1['collect'].corr(df_1['funded_amount'])

Как мы можем заметить, никаких зависимостей пока выявить нельзя. Попробую укрупнить группы по сумме кредита. Создам больше групп для заявок с небольшой суммой так как их количество больше.

In [ ]:
def funded_am_group(row):
    funded_amount = row['funded_amount']
    if funded_amount <= 100:
        return '0: 0-100'
    if 100 < funded_amount <= 200:
        return '1: 100-200'
    if 200 < funded_amount <= 300:
        return '2: 200-300'
    if 300 < funded_amount <= 400:
        return '3: 300-400'
    if 400 < funded_amount <= 500:
        return '4: 400-500'
    if 500 < funded_amount <= 700:
        return '5: 500-700'
    if 700 < funded_amount <= 1000:
        return '6: 700-1000'
    if 1000 < funded_amount <= 1300:
        return '7: 1000-1300'
    if 1300 < funded_amount <= 1500:
        return '8: 1300-1500'
    if funded_amount > 1500 :
        return '9: >1500'
df_1['funded_am_group'] = df_1.apply(funded_am_group, axis = 1)
df_2 = df_1.query('0 < collect < 60').sort_values(by='funded_am_group', ascending=True)
pivot = df_2.groupby('funded_am_group')['collect'].median().reset_index()
pivot.sort_values(by='collect', ascending=True)

По получившейся сводной таблице видно среднее количество дней необходимое для рассматриваемой суммы кредита. Построю график для болшей наглядности, очисив от выбросов.

In [ ]:
df_2 = df_1.query('0 < collect < 60').sort_values(by='funded_am_group', ascending=True)
fig = px.box(df_2, x="funded_am_group", y="collect", title='Cкорость сбора средств в зависимости от суммы кредита')
fig.show()

Как мы можем заметить, средние значения по всем группам говорят о том, что зависимость между количеством дней ,необходимых для сбора средств, и суммой кредита прямая. Это говорит о том, что в среднем относительно небольшие суммы собираются быстрее, в то время как на для больших сумм требуется больше времени.

**Зависимость скорости сбора средств от сектора и деятельности**
<a id="5"></a>

Проанализирую зависимость количества дней, необходимых для сбора суммы, и сектора заявки. По аналогии рассмотрю среднее количество дней для каждой категории. Для большей наглядности выбросов, минимальных и максимальных значений добавлена визуализация боксчартом.

In [ ]:
pivot_sector = df_2.groupby('sector')['collect'].median().reset_index()
pivot_sector.sort_values(by='collect', ascending=True)
fig = px.box(df_2, x="sector", y="collect", title='Cкорость сбора средств в зависимости от сектора', color_discrete_sequence=px.colors.qualitative.Dark24)
fig.show()

Можно заметить, что в среднем заявки с категориями Arts, Education и Manufacturing для которых нужно 6 дней , в то время как максимальное количество дней отмечается у таких секторов как  Transportation, Housing и Clothing для которых требуется как правило в среднем 12 дней на сборы.

Проанализируем деятельность.

In [ ]:
activity = df_1.groupby('activity')['id'].count().reset_index()
activity.sort_values(by='id', ascending=False).head(10)
top_activity = df_1[df_1['activity'].isin(['Farming', 'General Store', 'Pigs', 'Personal Housing Expenses', 'Food Production/Sales'])]
top_activity = top_activity.query('0 < collect < 60')
plt.figure(figsize=(17, 8))
ax = sns.lineplot(x='collect', y='funded_amount', hue='activity', data=top_activity) 
plt.show()

Начнем с того, что лидером по количеству заявкок является сельское хозяйство. В целом, в топ 10 входят заявки по темам связанным с сельским хозяйством, открытием нового магазина, реставрации дома и получением высшего образования.

Как показывает диаграмма, период можно разделить на 2 части. Тут отслеживается что среди заявок, удволетворенных в течение первых 40 дней, самая высокая скорость отслеживается в таких категориях как Farming, Personal Housing Expenses и Food Production/Sales, при том что во второй части динамики отмечается смена лидерских позиций на Pigs и Farming.

Интересно было бы рассмотреть за какой период времени в среднем заявки по 10 главным категориям удволетворяются.

In [ ]:
top10_activity = df_1[df_1['activity'].isin(['Farming', 'General Store', 'Pigs', 'Personal Housing Expenses', 'Food Production/Sales', 'Agriculture' ,'Retail', 'Home Appliances', 'Clothing Sales', 'Higher education costs'])]
top10_activity = top10_activity.query('0 < collect < 60')
top10_activity_sector = top10_activity.groupby('activity')['collect'].median().reset_index()
top10_activity_sector.sort_values(by='collect')

Как показывает полученная сводная таблица, не опираясь на сумму сбора, минимальное количество дней для закрытия заявки требуется для таких категорий как Home Appliances, Higher education costs и Pigs, в то время как максимальное количество дней требуется для заявок с деятельностью Agriculture и Clothing Sales.

**Зависимость скорости сбора средств от партнера**
<a id="6"></a>

Действуем по аналогии с предыдущим пунктом.

In [ ]:
top10_partners = df_2.groupby('partner_id')['id'].count().reset_index().sort_values(by='id', ascending=False).head(10)['partner_id']
df_1['funded_time'] = pd.to_datetime(df_1['funded_time'])
df_1['posted_time'] = pd.to_datetime(df_1['posted_time'])
df_1['time_funded'] = df_1['funded_time'] - df_2['posted_time']
df_1['hours_funded'] = df_1['time_funded'].dt.days * 24 + df_1['time_funded'].dt.seconds / 3600
df_1['dollar_per_hour'] = df_1['funded_amount']/df_1['hours_funded']
df_1['dollar_per_hour'] = df_1['dollar_per_hour'].round(2)
df_2 = df_1.query('0 < collect < 60')
# Фильтруем исходный DataFrame по полученным топ-10 партнерам
top10_partner_id = df_2[df_2['partner_id'].isin(top10_partners)]

top10_partner_id['partner_id'] = top10_partner_id['partner_id'].astype(str)
top10_partner_id = top10_partner_id.groupby('partner_id')['dollar_per_hour'].median().reset_index().sort_values(by='dollar_per_hour', ascending=False)

fg = px.bar(top10_partner_id, x='partner_id', y='dollar_per_hour', title='Средняя скорость удволетворения заявки в зависимости от партнера', color_discrete_sequence=px.colors.sequential.Plotly3)
fg.show()

Полученный график иллюстрирует среднюю скорость набора средств у тех партнеров, которые выдавали больше всего кредитов. Самая высокая скорость отмечается у партнера с номером 145(почти 1.6 долларов в час), 199(1.4 доллар в час) и 126(1.37 доллар в час), в то время как наименьшие результаты среди лидеров отмечаются у партнеров с айди 81(0.81 доллар в час), 156(0.86 доллар в час) и 311(0.94 доллар в час).

**Зависимость скорости сбора средств от страны, в которой был выдан кредит**

<a id="7"></a>

In [ ]:
df_2.groupby('country')['id'].count().reset_index().sort_values(by='id', ascending=False)
#Так как стран в нашем датасете 85, для отслеживания закономерностей стоит объединить их в определенные группы, например, разбить страны по частям света.
asia = ['Pakistan', 'Philippines', 'Cambodia', 'Iraq', 'Palestine', 'Mongolia', 'Tajikistan', 'Yemen', 'Vietnam', 'Jordan', 'Timor-Leste', 'Indonesia', 'Azerbaijan', 'Kyrgyzstan', 'Lebanon', 'India', 'Nepal', 'Israel', 'Thailand', 'Lao Peoples Democratic Republic', 'Myanmar (Burma)', 'South Sudan', 'China']
africa = ['Kenya', 'Senegal', 'Liberia', 'Tanzania', 'Mali', 'Ghana', 'Sierra Leone', 'Uganda', 'Burundi', 'Burkina Faso', 'The Democratic Republic of the Congo', 'Somalia', 'Zimbabwe', 'Cameroon', 'Rwanda', 'Zambia', 'Mozambique', 'Togo', 'Benin', 'Nigeria', 'Congo', 'Malawi', 'Egypt', 'South Africa', 'Madagascar', 'Lesotho']
europe = ['Georgia', 'Kosovo', 'Ukraine', 'Turkey', 'Armenia', 'Albania', 'Moldova']
north_america = ['Nicaragua', 'El Salvador', 'Honduras', 'Haiti', 'Guatemala', 'Costa Rica', 'Mexico', 'United States', 'Belize', 'Saint Vincent and the Grenadines', 'Dominican Republic', 'Panama', 'Puerto Rico']
oceania = ['Colombia', 'Samoa', 'Solomon Islands']
south_america = ['Peru', 'Ecuador', 'Bolivia', 'Brazil', 'Paraguay', 'Suriname']
def world_region(row):
    if row in asia:
        return 'Asia'
    if row in africa:
        return 'Africa'
    if row in north_america:
        return 'North America'
    if row in oceania:
        return 'Oceania'
    if row in south_america:
        return 'South America'
df_2['world_region'] = df_2['country'].apply(world_region)
df_2.groupby('world_region')['id'].count().reset_index().sort_values(by='id', ascending=False)

Как мы видим, больше всего заявок из Азии и Африки.

In [ ]:
plt.figure(figsize=(17, 6))
axxx = sns.lineplot(x='collect', y='funded_amount', hue='world_region', data=df_2) 
plt.show()

Как видно по получившемуся графику, график имеет свойство менять тенденцию примерно после отметки в 40 дней. Если в первой части графика заявки из Южной и Северной Америки имели самую высокую скорость сборов, в то время как другие страны имели примерно одинаковые показатели. Однако, после 40 дня ситуация меняется, и явно отстующей по скорости сбора страной становится Африка, в то время как страны Азии увеличили свои показатели, деля лидерство с Южной Америкой. Стоит отметить, что заявки из Африки стабильно медленно набирают необходимую сумму.

**Анализ заявок и индекса бедности**
<a id="8"></a>

Аггрегирую данные по заявкам в разрезе стран, сохраняя информацию о средних и суммарных показателях и объединю данные индекса бедности по странам с получившимся набором данных Kiva.

In [ ]:
#подгружаем данные о индексе бедности
mpi_index = {
 'Afghanistan':0.27172124, 'Albania':0.00274788, 'Algeria':0.0054091, 'Angola':0.28243506, 'Argentina':0.0014693, 
 'Armenia':0.000690069007857107, 'Azerbaijan':0.021,'Bangladesh':0.1040603, 'Belize':0.01710883, 'Benin':0.36767483, 
 'Bolivia':0.0377543, 'Botswana':0.0726387, 'Burundi':0.408861094240289, 
 'Cameroon':0.2320601, 'Central African Republic':0.4613475, 'Chad':0.5170112, 'Chile':0.207, 'Colombia':0.01965727, 
 'Congo':0.3311887, 'Costa Rica':0.0020063, 'Cote D\'Ivoire':0.235871, 
 'Cuba':0.0026887, 'Dominican Republic':0.0087862, 'Ecuador':0.0079374, 'Ethiopia':0.3666042, 'Gambia':0.1980231, 
 'Georgia':0.0012446, 'Ghana':0.1112183, 'Guinea':0.3732216, 'Guinea-Bissau':0.3406887, 'Guyana':0.0065924, 
 'Haiti':0.1995877, 'Honduras':0.0511542, 'India':0.0688106, 'Indonesia':0.0140107, 'Iraq':0.03269432, 
 'Israel':None,'Jamaica':0.0108103, 'Jordan':0.00152592, 'Kiribati':0.0801574, 'Kyrgyzstan':0.001426, 
 'Lao People\'s Democratic Republic':0.10833325, 'Lebanon':0.531,'Lesotho':0.0843592, 'Liberia':0.2592937, 'Madagascar':0.3839745, 
 'Malawi':0.2310952, 'Maldives':0.00265409, 'Mali':0.3760629, 'Mauritania':0.3270372, 'Mexico':0.0280538, 
 'Mongolia':0.0281268, 'Montenegro':0.0048989, 'Morocco':0.0266967, 'Myanmar (Burma)':0.17584623, 'Nepal':0.0743989, 
 'Nigeria':0.2543896, 'North Macedonia':0.0014221, 'Pakistan':0.1982474, 'Palestine':0.0019801, 'Panama':0.191,
 'Papua New Guinea':0.2632909, 'Paraguay':0.01884858, 'Peru':0.0292211, 'Philippines':0.02424934, 'Rwanda':0.231002, 
 'Samoa':0.0246005, 'Sao Tome and Principe':0.0479234, 'Senegal':0.262862, 'Serbia':0.0004331, 'Seychelles':0.0029635, 
 'Sierra Leone':0.2928993, 'Somalia':0.514,'South Africa':0.02489064, 'Sri Lanka':0.0111847, 'Suriname':0.0112325, 
 'Tajikistan':0.02900592, 'Tanzania':0.284179310669258, 'Thailand':0.0021207, 'Timor-Leste':0.221514240070676,
 'Togo':0.1796163, 'Tonga':0.0033362, 'Tunisia':0.0028877, 'Turkmenistan':0.0008492, 'Turkey':0.028,
 'Tuvalu':0.0080846, 'Uganda':0.281028478426768, 'Vietnam':0.0077294, 'Zambia':0.2316851, 'Zimbabwe':0.1099418, 
 'Barbados':0.00852886, 'Bhutan':0.17486399, 'Bosnia and Herzegovina':0.0083075, 'Brazil':0.01634604, 
 'Burkina Faso':0.523424279357662, 'Cambodia':0.17034812, 'China':0.0160667, 'Comoros':0.18077141, 
 'The Democratic Republic of the Congo':0.1116763, 
 'Egypt':0.0196817970481763, 'El Salvador':0.03246251, 'Eswatini (Kingdom of)':0.08127132, 'Gabon':0.0696953622816103, 
 'Guatemala':0.13351782, 'Kazakhstan':0.00161063266199911, 'Kenya':0.170776074964141, 'Libya':0.00742146512493491, 
 'Moldova':0.00353391, 'Mozambique':0.416955412151251, 'Namibia':0.184734534885262, 
 'Nicaragua':0.0744948916698962, 'Niger':0.601279812220296, 'Saint Lucia':0.00720186, 'South Sudan':0.58015746, 
 'Sudan':0.2794396, 'Trinidad and Tobago':0.00241792, 'Vanuatu':0.129,'Ukraine':0.000840431758839254, 
 'Yemen':0.245166461457986 }

In [ ]:
df_mpi_index = pd.DataFrame.from_dict(mpi_index,orient = 'index').reset_index()
df_mpi_index = df_mpi_index.rename(columns={'index': 'country', 0: 'mpi_index'})
df_mpi_index

In [ ]:
df_new = df_1[['country', 'id', 'funded_amount', 'loan_amount', 'partner_id', 'term_in_months', 'lender_count', 'collect', 'hours_funded', 'dollar_per_hour']]
index_df = df_new.groupby('country').agg('median').reset_index()
index_df

In [ ]:
df_mpi = df_mpi_index.merge(index_df, on='country', how='right')
df_mpi

Так как в датасете представлено слишком много стран, я буду рассматривать 15 стран с самым высоком показателем бедности.

In [ ]:
min_mpi_index = df_mpi.sort_values(by='mpi_index', ascending=False)
min_mpi_index.head(15)

In [ ]:
poorest_countries = ['South Sudan', 'Lebanon', 'Burkina Faso', 'Somalia', 'Mozambique', 'Burundi', 'Madagascar', 'Mali', 'Benin', 'Congo', 'Sierra Leone', 'Tanzania', 'Uganda', 'Senegal', 'Liberia']
poorest_countries_df = df_1[df_1['country'].isin(poorest_countries)]
poorest_countries_df

In [ ]:
#Рассмотрим общее распределение заявок по секторам среди 15 самых бедных стран.
poorest_countries_df_sector = poorest_countries_df[['sector', 'country', 'id']]
pr = poorest_countries_df_sector.groupby(['sector', 'country']).count().reset_index()
fig_2 = px.bar(pr, x="sector", y="id", color="country", title="Распределение заявок по секторам среди беднейших стран по странам")
fig_2.show()

In [ ]:
pp = poorest_countries_df.groupby('sector')['id'].count().reset_index()

axi = px.bar(pp, x='sector', y='id', title='Распределение заявок по секторам среди беднейших стран')
axi.show()

Такие категории как еда, сельское хозяйство и розничная торговля являются самыми популярными категорями среди заявок в 15 беднейших странах, в то время как развлечения, розничная торговля, искусство и производство реже всего являются темой заявки.


Рассмотрю распределение секторов в заявках в 4 беднейших стран из топа.

In [ ]:
poorest1_countries = ['South Sudan']
poorest1_countries = df_1[df_1['country'].isin(poorest1_countries)]
poorest1_countries = poorest1_countries.groupby('sector')['id'].count().reset_index()
poorest3_countries = ['Lebanon']
poorest3_countries = df_1[df_1['country'].isin(poorest3_countries)]
poorest3_countries = poorest3_countries.groupby('sector')['id'].count().reset_index()
poorest2_countries = ['Burkina Faso']
poorest2_countries = df_1[df_1['country'].isin(poorest2_countries)]
poorest2_countries = poorest2_countries.groupby('sector')['id'].count().reset_index()
poorest4_countries = ['Somalia']
poorest4_countries = df_1[df_1['country'].isin(poorest4_countries)]
poorest4_countries = poorest4_countries.groupby('sector')['id'].count().reset_index()
specs = [[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=2, cols=2, specs=specs)
fig.add_trace(go.Pie(labels=['Agriculture', 'Arts', 'Clothing', 'Construction', 'Education', 'Food', 'Health', 'Housing', 'Manufacturing', 'Retail', 'Services'], values=[13, 2, 7, 2, 6, 35, 7, 12, 1, 61, 13], name="South Sudan"), 1, 1)
fig.add_trace(go.Pie(labels=['Agriculture', 'Arts', 'Clothing', 'Construction', 'Food', 'Health', 'Manufacturing', 'Personal Use', 'Retail', 'Services', 'Transportation'], values=[530, 47, 269, 10, 852, 4, 29, 4, 356, 92, 1], name="Burkina Faso"),1, 2)
fig.add_trace(go.Pie(labels=['Agriculture', 'Arts', 'Clothing', 'Construction', 'Education', 'Entertainment',  'Food', 'Health', 'Housing',  'Manufacturing', 'Personal Use', 'Retail', 'Services', 'Transportation', 'Wholesale'], values=[224, 113, 473, 154, 2294, 27, 962, 473, 232, 112
, 385, 652, 1093, 108, 7], name="Lebanon"),2, 1)
fig.add_trace(go.Pie(labels=['Agriculture', 'Construction', 'Education', 'Food', 'Health', 'Manufacturing', 'Personal Use', 'Retail', 'Services', 'Transportation'], values=[2, 1, 14, 7, 2, 1, 1, 13, 1, 1], name="Somalia"),2, 2)
fig.update(layout_title_text='Распределение секторов по заявкам в 4 беднейших странах',
           layout_showlegend=False)

fig.show()

Мы можем заметить, что в целом ключевые категории повторяются в топе беднейших стран. Среди них retail, food, agriculture и education являются самыми распространенными среди таких стран как Южный Судан, Буркина-Фасо, Ливан и Сомали.

Рассмотрю заявки в разрезе тем. По аналогии рассмотрю 10 самых популярных из-за огромного списка в датасете.

In [ ]:
activity10 = poorest_countries_df[poorest_countries_df['activity'].isin(['Farming', 'General Store', 'Pigs', 'Personal Housing Expenses', 'Food Production/Sales', 'Agriculture' ,'Retail', 'Home Appliances', 'Clothing Sales', 'Higher education costs'])]
poorest_countries_df_sector = poorest_countries_df[['activity', 'country', 'id']]
prp = activity10.groupby(['activity', 'country']).count().reset_index()
fig_2 = px.bar(prp, x="activity", y="id", color="country", title="Распределение стран по темам заявок")
fig_2.show()

Как мы можем увидеть, среди 10 самых бедных стран такие темы заявок как фермерство(4000), розничная торговля(3342) и универмаг(3053) являются самыми распространенными, в то время как свиньи(580), траты на высшее образование(645) и приборы для дома(97) реже всего упомянались в заявках.
В теме фермерство больше всего заявок из Уганды и Мали; в теме розничная торговля превалируют заявки из Либерии, Уганды и Сенегала; в теме универмаг заявки из Уганды занимают существенную пропорцию, а также много заявок из Сьерра-Леоне и Мозамбик.

In [ ]:
rr_1 = activity10.groupby('activity')['funded_amount'].sum().reset_index()
rr_1 = px.line(rr_1, x='activity', y='funded_amount', title='Сумма кредитов по темам среди беднейших стран')
rr_1.show() 


Можно отметить, что среди самых бедных по индексу стран наибольшие средства выделяются на погашение заявок с такими темами как фермерство(2.7 миллионов долларов) и универмаг(1.8 миллионов долларов), что повторяет тенденцию графика выше, однако также большие средства идут на производство еды и продажи(2.06 миллиона долларов) а также магазин одежды(1.74 миллионов долларо).

In [ ]:
for_pivot = df_1.groupby('country')['id'].count()
df_mpi_new = df_mpi_index.merge(for_pivot, on='country', how='left')
poorest = df_mpi_new.sort_values(by='mpi_index', ascending=False).head(15)
poorest

Получившаяся таблица показывает нам, что в многих беднейших странах в совсем отсутствует платформа Кива. В 15 самых бедных стран наибольшее количество заявок удволетворено в Ливане, Мали и Мадагаскаре. Вообще представленный список иллюстрирует, что платформа Кива не представлена в достаточно большом количестве бедных стран, хотя уровень достатка предполагает нужну в подобного рода сервисах, например в Нигере,Чаде и Центральной Африканской республике.

**Распределение женщин-заемщиков в разных странах и религиях**
<a id="9"></a>

создам столбец с долей женщин-заемщиков на основе стообца borrower_genders.

In [ ]:
df_1['all'] = df_1['borrower_genders'].str.count('male') 
df_1['female'] = df_1['borrower_genders'].str.count('female')
df_1['female_proportion'] = df_1['female']/df_1['all']
def female_proportion_group(row):
    female_proportion = row['female_proportion']
    if 0.0 < female_proportion < 0.1:
        return '0-10%'
    if female_proportion == 0.0:
        return '0%'
    if 0.1 < female_proportion <= 0.2:
        return '10%-20%'
    if 0.2 < female_proportion <= 0.3:
        return '20%-30%'
    if 0.3 < female_proportion <= 0.4:
        return '30%-40%'
    if 0.4 < female_proportion <= 0.5:
        return '40%-50%'
    if 0.5 < female_proportion <= 0.6:
        return '50%-60%'
    if 0.6 < female_proportion <= 0.7:
        return '60%-70%'
    if 0.7 < female_proportion <= 0.8:
        return '70%-80%'
    if 0.8 < female_proportion <= 0.9:
        return '80%-90%'
    if 0.9 < female_proportion < 1.0:
        return '90%-100%'
    if female_proportion == 1.0 :
        return '100%'
df_1['female_proportion_group'] = df_1.apply(female_proportion_group, axis = 1)
cattegory = df_1.groupby('female_proportion_group')['id'].count().reset_index().sort_values(by='id', ascending=False)
figx = px.pie(cattegory, values='id', names='female_proportion_group', title='Распределение заявок по категориям женщин-заемщиков', color_discrete_sequence=px.colors.sequential.Plotly3)
figx.show() 


Полученный пайчарт показывает, что заемщиками 75 процентов заявок являются только женщины(432805 заявок), в то время почти 20 процентов заявок были созданы только мужчинами-заемщиками(108621 заявок). На третьем месте в этом топе содержатся заявки с процентным содержанием 70-80%% женщин(6037 заявок).

Расмотрю распределение заявок по религиям. Проанализирую долю заемщиков по основным трем религиям.

In [ ]:
main_religion = df_1.groupby('main_country_religion')['id'].count().reset_index().sort_values(by='id', ascending=False)
fig, ax = plt.subplots(figsize=(6,4))
main_rel = ['Christians','Muslims','Buddhists', 'Folk Religions']
sns.kdeplot(data=df_1.query('main_country_religion in @main_rel'),
hue='main_country_religion',x='female_proportion',common_norm=False,bw_adjust=1/2);

На основе общего датасета мы можем заметить, что больше всего заявок из христианских стран(428550 заявок) и из мусульманских стран(90904), в то время как меньше всего из еврейских стран(30).

Полученный график показывает, что заявки составленные только мужчинами были в большей степени у стран с основной религией христианство и чуть меньшее количество с религией ислам, в то время как в обратной ситуации, когда 100 процентов заемщиков это женщины, в основными религиями является христианство и народная религия. При этом на остальных уровнях, когда заемщики и мужчины и женищны, заявки были из стран с доминирующей религией буддизм.

In [ ]:
category = df_1.groupby('female_proportion_group')['collect'].median().reset_index().sort_values(by='collect', ascending=False)
categoryy = px.line(category, x='female_proportion_group', y='collect', title='Средняя скорость сбора средств в зависимости от доли женщин-заемщиков')
categoryy.show() 

In [ ]:
fig = px.box(df_2.query('main_country_religion in @main_rel'), x="main_country_religion", y="collect", title='Cкорость сбора средств в зависимости от главной религии', color_discrete_sequence=px.colors.qualitative.Dark24)
fig.show()

Как показывают получившиеся диаграммы, заявки из мусульманских страны в среднем дольше собирают деньги на свои нужды, за 12 дней, а также максимальный результат, 59 дней, принадлежит именно этой группе, а буддистские наоборот в среднем всего за 8 дней удволетворяют заявки.

Также важно учесть, что в среднем чисто женские заявки намного быстрее собирают необходимую сумму, чем мужские.

**Партнёрские отношения Kiva в регионах**
<a id="10"></a>

В одном из пунктов выше мы рассмотрели тех партнеров, которые погасили больше всего заявок. Сейчас рассмотрю топ 10 партнеров, которые выдали самую большую сумму в помощь нуждающимся.

In [ ]:
top10_partner_id_tab = df_1.groupby('partner_id')['loan_amount'].sum().reset_index().sort_values(by='loan_amount', ascending=False)
top10_partner_id_tab = top10_partner_id_tab.head(10)
top10_partner_id_tab['partner_id'] = top10_partner_id_tab['partner_id'].astype(str)

axi = px.bar(top10_partner_id_tab, x='partner_id', y='loan_amount', title='Топ 10 партнеров по объему выданных средств')
axi.show()

Мы можем увидеть, что топ 10 партнеров по вложенной сумме отличается от топ 10 по количеству удволетворенных заявок. Это связано с тем, что некоторые партнеры предпочитали закрывать заявки на большие суммы, чем другие.
Партнер с айди 145 является явным лидером по выданным кредитам(33.51 миллиона долларов), следом идет партнер 199(10.4 миллионов долларов) и 177(7.5 миллионов долларов).

Рассмотрим количество партнёров Kiva в разных странах.


In [ ]:
partner_in_con = df_1.groupby('country')['partner_id'].count().reset_index().sort_values(by='partner_id', ascending=False)
partner_in_conn = partner_in_con.head(5)
axixi = px.bar(partner_in_conn, x='country', y='partner_id', title='Топ-5 стран по количеству партнеров')
axixi.show()

In [ ]:
partner_in_conm = partner_in_con.tail(5)
axixix = px.bar(partner_in_conm, x='country', y='partner_id', title='5 стран c наименьшим количеству партнеров')
axixix.show()

Количество партнеров по странам повторяет тенденции, выявленные в одном из пунктов выше, когда мы составляли список с количеством заявок по странам, поэтому делаю вывод что это два взаимосвязанных фактора.

Выявим частоту и объем случаев, когда партнеры выдают кредиты до полного финансирования.
Разделим заявки на те, которые сперва были выплачены партнерами заемщику(disbursed_time), а потом собраны(funded_time) и вторую группу, где были сперва собраны(funded_time), а потом выплачены (t_disbursed_time). Где день совпадает объединю в одну группу.

In [ ]:
df_1['delta'] = (df_1['disbursed_time'] - df_1['funded_time']).dt.days
def credit_group(row):
    delta = row['delta']
    if delta < 0:
        return 'group 1'
    if delta > 0:
        return 'group 2'
    if delta == 0:
        return 'group 3'
df_1['credit_group'] = df_1.apply(credit_group, axis = 1)

credit_group = df_1.groupby('credit_group')['id'].count().reset_index()
credit_group

In [ ]:
figxg = px.pie(credit_group, values='id', names='credit_group', title='Распределение партнеров в зависимости порядка выдачи кредит')
figxg.show() 

In [ ]:
kr = df_1['partner_id'].value_counts().reset_index()
len(kr['count'])

In [ ]:
credit_group1 = df_1[df_1['credit_group'].isin(['group 1'])]
credit_group_part_1 = credit_group1['partner_id'].value_counts().reset_index()
len(credit_group_part_1[['count']])

Как показывают получившиеся чарты и таблица, 95 процентов заявок удволетворяются до полного сбора средств, что говорит о том что кива имеет большую базу доверяющих платформе. 251 партнер из 278 выдают кредиты до полного финансирования.

In [ ]:
pie = credit_group1.groupby('country')['partner_id'].count().reset_index()
f = px.pie(pie, values='partner_id', names='country', title='Партнеры в разрезе стран среди заявок, которые были выданы до полного финансирования')
f.update_traces(textposition='inside')
f.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
f.show()

Среди тех заявок, которые были удволетворены до полного финансирования, партнеры в 30 процентов случаев из Филиппин, 10 процентов кредитов из Кении и почти 6 процентов из Сальвадоре.

In [ ]:
pie = credit_group1.groupby('country')['loan_amount'].sum().reset_index()
fa = px.pie(pie, values='loan_amount', names='country', title='Партнеры в разрезе стран среди заявок, которые были выданы до полного финансирования')
fa.update_traces(textposition='inside')
fa.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fa.show()

Объем средств, выданных до полного финансирования для удволетворения заявок, распределился следующим образом: 16 процентов финансов из Филиппин(52,818,900); 7.3 процентов из Кении(22,997,150) и из Сальвадора(19,895,575).

**Общие выводы**
<a id="11"></a>

Зависимость между суммой кредита и временем сбора:

*  Средняя сумма кредита растет пропорционально времени, необходимому для ее сбора.  
*  Это означает, что маленькие суммы собираются быстрее, чем большие.

Категории и время сбора:

*  Категории Arts, Education и Manufacturing обычно собираются за 6 дней.
*  Транспорт, жилье и одежда требуют в среднем 12 дней.

Популярные категории:

*  Сельское хозяйство - самая популярная категория.
*  В топ-10 входят также: сельское хозяйство, открытие магазина, ремонт дома, высшее образование.

Скорость сбора по партнерам:

*  Партнеры 145, 199 и 126 демонстрируют самую высокую скорость сбора.
*  Партнеры 81, 156 и 311 имеют самые низкие показатели.

География:

*  Филиппины - страна с наибольшим количеством заявок.
*  Конго - страна с наименьшим количеством заявок.
*  Азия и Африка - регионы с наибольшим числом заявок.

Анализ по беднейшим странам:

*  Еда, сельское хозяйство и розничная торговля - самые популярные категории в 15 беднейших странах.
*  Развлечения, искусство и производство - самые редкие категории.
*  Фермерство, розничная торговля и универмаги - самые распространенные темы заявок в 10 самых бедных странах.
*  Заявки на фермерство наиболее распространены в Уганде и Мали.
*  Заявки на розничную торговлю преобладают в Либерии, Уганде и Сенегале.
*  Заявки на универмаги часты в Уганде, Сьерра-Леоне и Мозамбике.
*  Ливан, Мали и Мадагаскар - страны с наибольшим количеством удовлетворенных заявок.
*  Kiva представлена не во всех бедных странах (например, Нигер, Чад, Центральноафриканская республика).

Анализ по доле заемщиков:

*  75% заявок подают женщины.
*  20% заявок подают мужчины.
*  Заявки с мужчинами-заемщиками чаще встречаются в странах с преобладающим христианством и исламом.
*  Заявки с женщинами-заемщиками чаще встречаются в странах с преобладающим христианством и традиционными религиями.
*  Заявки, поданные только женщинами,  собирают деньги быстрее, чем заявки только мужчин. 
*  Мусульманские страны, как правило, собирают средства дольше (в среднем 12 дней).
*  Буддийские страны собирают средства быстрее (в среднем 8 дней).

Анализ по партнерам:

*  Партнер 145 - лидер по количеству выданных кредитов. 
*  95% заявок удовлетворяются до полного финансирования, что свидетельствует о доверии к платформе Kiva.

Общее:

*  Данные показывают, что Kiva активно используется в развивающихся странах, особенно в Азии и Африке.
*  Платформа пользуется популярностью среди женщин-заемщиков.
*  Kiva эффективно помогает людям собирать средства на различные нужды.